In [1]:
import pandas as pd

In [10]:
data = pd.read_csv('1988.csv', header = 0, dtype = str)
data.head()

,STU_ID,F4UNIV1,F4UNI2A,F4UNI2B,F4UNI2C,F4UNI2D,F4UNI2E,BYS2A,BYS4A,BYS4OCC,...,F4STATUS,F4SGPAR,F4HSGRDT,F4HSDIPL,F4HSTYPE,F4HHDG,F4TYPEDG,F4ATT4YR,F4ATTPSE,F4ENRL00
0,124902,1126,1,1,1,1,1,1,1,15,...,1,-3,199206,1,1,1,1,2,1,2
1,124915,1126,1,1,1,1,1,1,1,14,...,3,1,199206,1,1,1,1,1,1,1
2,124916,1126,1,1,1,1,1,1,1,15,...,3,-3,199206,1,1,4,4,1,1,1
3,124932,1126,1,1,1,1,1,1,1,15,...,1,-3,199206,1,1,3,3,2,1,2
4,124944,1126,1,1,1,1,1,1,1,1,...,1,-3,199206,1,1,4,4,1,1,2


In [11]:
data.info

<bound method DataFrame.info of         STU_ID F4UNIV1 F4UNI2A F4UNI2B F4UNI2C F4UNI2D F4UNI2E BYS2A BYS4A  \
0       124902    1126       1       1       1       1       1     1     1   
1       124915    1126       1       1       1       1       1     1     1   
2       124916    1126       1       1       1       1       1     1     1   
3       124932    1126       1       1       1       1       1     1     1   
4       124944    1126       1       1       1       1       1     1     1   
...        ...     ...     ...     ...     ...     ...     ...   ...   ...   
12139  9199119    1126       1       1       1       1       1     1     1   
12140  9199152    1126       1       1       1       1       1     1     1   
12141  9199157    1126       1       1       1       1       1     1     1   
12142  9199179    1126       1       1       1       1       1     1     1   
12143  9199197    1126       1       1       1       1       1     1     1   

      BYS4OCC  ... F4STATUS F4S

In [12]:
data = pd.read_csv('1994.csv', header = 0, dtype = str)
data.head()

,STU_ID,F3UNIV1,F3UNIV2A,F3UNIV2B,F3UNIV2C,F3UNIV2D,F3UNIV2E,SCH_ID,SSTRATID,BYS2A,...,OCCVERB3,OCCSRCH3,INDVERB1,INDSRCH1,INDVERB2,INDSRCH2,INDVERB3,INDSRCH3,OCCFUTVB,OCCFUTFS
0,124902,126,1,1,1,1,1,1249,1,1,...,COOK,COOK,-9,-9,-6,-6,RESTAURANT,RESTAURANT,CHEF,CHEF
1,124915,126,1,1,1,1,1,1249,1,1,...,-6,-6,-6,-6,-6,-6,-6,-6,ENGINEERING MECHANICAL,ENGINEERING MECHANICAL
2,124916,126,1,1,1,1,1,1249,1,1,...,-6,-6,-6,-6,-6,-6,-6,-6,MEDICAL X RAY TECHNICIAN RADIOLOGIST,MEDICAL XRAY RESPIRATORY TECHNICIAN RADIOLOGIST
3,124932,126,1,1,1,1,1,1249,1,1,...,-6,-6,-6,-6,-6,-6,-6,-6,PARA LEGAL,*PARA LEGAL
4,124944,126,1,1,1,1,1,1249,1,1,...,-6,-6,-6,-6,-6,-6,-6,-6,TEACHER,TEACHER


In [ ]:
data = pd.read_csv('1992.csv', header = 0, dtype = str)
data.head()